In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

22/03/22 17:25:18 WARN Utils: Your hostname, MacBook-Air-de-Aitor.local resolves to a loopback address: 127.0.0.1; using 172.20.10.2 instead (on interface en0)
22/03/22 17:25:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/22 17:25:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Carga los logs de la carpeta weblogs

In [2]:
df = spark.read.text("weblogs/2014-01-18.log")
df.printSchema()
df.describe().show()

root
 |-- value: string (nullable = true)



+-------+--------------------+
|summary|               value|
+-------+--------------------+
|  count|                6404|
|   mean|                null|
| stddev|                null|
|    min|                    |
|    max|99.73.191.11 - 20...|
+-------+--------------------+



In [10]:
logs=sc.textFile("weblogs/*.log")

2. Crea un RDD de pares a partir de estos ficheros con el usuario como clave y un valor 1

In [11]:
usuariosRDD = logs.map(lambda linea: linea.split()).map(lambda words: (words[2].strip(),1))
usuariosRDD.take(5)

[('128', 1), ('128', 1), ('94', 1), ('94', 1), ('131', 1)]

3. Crea un nuevo RDD con la suma de los valores para cada identificador de usuario

In [13]:
usuariosTotal = usuariosRDD.reduceByKey(lambda a,b: a+b)
usuariosTotal.take(5)

[('54126', 32), ('54', 498), ('21443', 35), ('74', 516), ('64', 562)]

4. Muestra los 10 usuarios que más peticiones han realizado.

In [17]:
usuariosTotal.takeOrdered(10, lambda x: -x[1])

[('193', 667),
 ('13', 636),
 ('24', 620),
 ('147', 610),
 ('110', 606),
 ('84', 606),
 ('111', 604),
 ('77', 603),
 ('21', 600),
 ('126', 600)]

Crea un nuevo RDD de pares donde la clave sea el código del usuario y el valor una lista de IP desde donde se ha conectado el usuario.  